# Bipartite Model for Concept Bottleneck

This notebook demonstrates how to:
1. Load and prepare data with rich concept annotations
2. Define concept and task metadata with distributions and cardinalities
3. Build a BipartiteModel that automatically constructs a ProbabilisticModel
4. Use LazyConstructors to create encoder and predictor factors
5. Train the model with concept and task supervision
6. Apply interventions within the BipartiteModel framework

## 1. Imports

We import the necessary libraries:
- **PyTorch**: for neural network building blocks and distributions
- **sklearn**: for evaluation metrics
- **torch_concepts**: for Annotations, BipartiteModel, LazyConstructors, and inference

In [49]:
import torch
from sklearn.metrics import accuracy_score
from torch.distributions import RelaxedOneHotCategorical, RelaxedBernoulli

from torch_concepts import Annotations, AxisAnnotation
from torch_concepts.data import ToyDataset
from torch_concepts.nn import (
    LinearZC,
    LinearCC,
    RandomPolicy,
    DoIntervention,
    intervention,
    DeterministicInference,
    BipartiteModel,
    LazyConstructor, UniformPolicy
)

## 2. Data Loading and Preparation

We load the XOR toy dataset and prepare the training data:
- **Features (x_train)**: input features for the model
- **Concepts (c_train)**: intermediate concept labels (binary: c1, c2)
- **Targets (y_train)**: task labels (converted to one-hot encoding with 2 classes)
- **Names**: concept and task attribute names

In [50]:
# Hyperparameters
latent_dims = 10
n_epochs = 500
n_samples = 1000
concept_reg = 0.5

# Load toy XOR dataset
data = ToyDataset('xor', size=n_samples, random_state=42)
x_train = data.data
c_train = data.concept_labels
y_train = data.target_labels
concept_names_raw = data.concept_attr_names
task_names_raw = data.task_attr_names

# Convert y_train to one-hot encoding (2 classes)
y_train = torch.cat([y_train, 1 - y_train], dim=1)

# Define concept and task names for the model
concept_names = ('c1', 'c2')
task_names = ('xor',)

print(f"Dataset loaded:")
print(f"  Features shape: {x_train.shape}")
print(f"  Concepts shape: {c_train.shape}")
print(f"  Targets shape: {y_train.shape}")
print(f"  Concept names: {concept_names}")
print(f"  Task names: {task_names}")

Dataset loaded:
  Features shape: torch.Size([1000, 2])
  Concepts shape: torch.Size([1000, 2])
  Targets shape: torch.Size([1000, 2])
  Concept names: ('c1', 'c2')
  Task names: ('xor',)


## 3. Rich Annotations with Metadata

The **Annotations** object in the BipartiteModel framework supports rich metadata:
- **Cardinalities**: The number of classes/dimensions for each variable
- **Metadata**: Additional information for each variable including:
  - **distribution**: The probability distribution type
  - **type**: Variable type (e.g., 'binary', 'categorical')
  - **description**: Human-readable description

This metadata is used by the BipartiteModel to automatically:
- Create appropriate Variables
- Set up correct probability distributions
- Configure the ProbabilisticModel structure

In [51]:
# Define cardinalities (number of classes for each variable)
cardinalities = (1, 1, 2)  # c1: 1 (binary), c2: 1 (binary), xor: 2 (one-hot)

# Define metadata for each variable
metadata = {
    'c1': {
        'distribution': RelaxedBernoulli, 
        'type': 'binary', 
        'description': 'Concept 1'
    },
    'c2': {
        'distribution': RelaxedBernoulli, 
        'type': 'binary', 
        'description': 'Concept 2'
    },
    'xor': {
        'distribution': RelaxedOneHotCategorical, 
        'type': 'binary', 
        'description': 'XOR Task'
    },
}

# Create rich annotations
annotations = Annotations({
    1: AxisAnnotation(
        concept_names + task_names, 
        cardinalities=cardinalities, 
        metadata=metadata
    )
})

print("Annotations structure:")
print(f"  Variables: {concept_names + task_names}")
print(f"  Cardinalities: {cardinalities}")
print(f"\nMetadata:")
for name, meta in metadata.items():
    print(f"  {name}:")
    print(f"    Distribution: {meta['distribution'].__name__}")
    print(f"    Type: {meta['type']}")
    print(f"    Description: {meta['description']}")

Annotations structure:
  Variables: ('c1', 'c2', 'xor')
  Cardinalities: (1, 1, 2)

Metadata:
  c1:
    Distribution: RelaxedBernoulli
    Type: binary
    Description: Concept 1
  c2:
    Distribution: RelaxedBernoulli
    Type: binary
    Description: Concept 2
  xor:
    Distribution: RelaxedOneHotCategorical
    Type: binary
    Description: XOR Task


## 4. BipartiteModel: High-Level Model Construction

The **BipartiteModel** is a high-level abstraction that:
- Automatically constructs a ProbabilisticModel from annotations
- Uses **LazyConstructors** to create encoder and predictor factors
- Manages the bipartite structure: concepts → tasks
- Exposes the underlying ProbabilisticModel for inference and interventions

### LazyConstructors:
- **LazyConstructor(LinearZC)**: Creates encoder factors for concepts
- **LazyConstructor(LinearCC)**: Creates predictor factors for tasks

The BipartiteModel automatically:
1. Creates Variables from annotations
2. Builds ParametricCPDs using LazyConstructors
3. Constructs the ProbabilisticModel with proper dependencies

In [52]:
# Create the encoder (input features -> embedding)
encoder = torch.nn.Sequential(
    torch.nn.Linear(x_train.shape[1], latent_dims), 
    torch.nn.LeakyReLU()
)

# Create the BipartiteModel
concept_model = BipartiteModel(
    task_names=task_names,
    input_size=latent_dims,
    annotations=annotations,
    encoder=LazyConstructor(LinearZC),
    predictor=LazyConstructor(LinearCC)
)

print("BipartiteModel structure:")
print(f"  Task names: {task_names}")
print(f"  Latent dimensions: {latent_dims}")
print(f"  Concept propagator: {LinearZC.__name__}")
print(f"  Task propagator: {LinearCC.__name__}")
print(f"\nUnderlying ProbabilisticModel:")
print(concept_model.probabilistic_model)
print(f"\nThe model automatically created:")
print(f"  - Variables for concepts and tasks")
print(f"  - Encoder factors (embedding → concepts)")
print(f"  - Predictor factors (concepts → tasks)")

BipartiteModel structure:
  Task names: ('xor',)
  Latent dimensions: 10
  Concept propagator: ProbEncoderFromEmb
  Task propagator: ProbPredictor

Underlying PGM:
ProbabilisticGraphicalModel(
  (factor_modules): ModuleDict(
    (embedding): Identity()
    (c1): ProbEncoderFromEmb(
      (encoder): Sequential(
        (0): Linear(in_features=10, out_features=1, bias=True)
        (1): Unflatten(dim=-1, unflattened_size=(1,))
      )
    )
    (c2): ProbEncoderFromEmb(
      (encoder): Sequential(
        (0): Linear(in_features=10, out_features=1, bias=True)
        (1): Unflatten(dim=-1, unflattened_size=(1,))
      )
    )
    (xor): ProbPredictor(
      (predictor): Sequential(
        (0): Linear(in_features=2, out_features=2, bias=True)
        (1): Unflatten(dim=-1, unflattened_size=(2,))
      )
    )
  )
)

The model automatically created:
  - Variables for concepts and tasks
  - Encoder factors (embedding → concepts)
  - Predictor factors (concepts → tasks)


## 5. Inference Engine

We use the **DeterministicInference** engine on the BipartiteModel's underlying ProbabilisticModel:
- **Evidence**: The embedding computed from input features
- **Query**: The concepts and tasks we want to infer

The BipartiteModel exposes its ProbabilisticModel via the `.probabilistic_model` attribute.

In [53]:
# Initialize the inference engine with the BipartiteModel's ProbabilisticModel
inference_engine = DeterministicInference(concept_model.probabilistic_model)

# Define the query (what we want to infer)
query_concepts = ["c1", "c2", "xor"]

print("Inference setup:")
print(f"  Engine: DeterministicInference")
print(f"  ProbabilisticModel source: concept_model.probabilistic_model")
print(f"  Query variables: {query_concepts}")
print(f"\nInference flow:")
print(f"  x_train → encoder → embedding → [c1, c2] → xor")

Inference setup:
  Engine: DeterministicInference
  PGM source: concept_model.pgm
  Query variables: ['c1', 'c2', 'xor']

Inference flow:
  x_train → encoder → embedding → [c1, c2] → xor


## 6. Complete Model Pipeline

We combine the encoder and BipartiteModel into a complete pipeline:
- **encoder**: Maps input features to latent embedding
- **concept_model**: BipartiteModel that maps embedding to concepts and tasks

This creates a Sequential model for easy training.

In [54]:
# Combine encoder and concept_model into a Sequential pipeline
model = torch.nn.Sequential(encoder, concept_model)

print("Complete model pipeline:")
print(model)
print(f"\nPipeline structure:")
print(f"  1. Encoder: {x_train.shape[1]} features → {latent_dims} dimensions")
print(f"  2. BipartiteModel: {latent_dims} dimensions → concepts & tasks")

Complete model pipeline:
Sequential(
  (0): Sequential(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (1): BipartiteModel(
    (_encoder_builder): Propagator(
      (module): ProbEncoderFromEmb(
        (encoder): Sequential(
          (0): Linear(in_features=10, out_features=1, bias=True)
          (1): Unflatten(dim=-1, unflattened_size=(1,))
        )
      )
    )
    (_predictor_builder): Propagator(
      (module): ProbPredictor(
        (predictor): Sequential(
          (0): Linear(in_features=2, out_features=2, bias=True)
          (1): Unflatten(dim=-1, unflattened_size=(2,))
        )
      )
    )
    (pgm): ProbabilisticGraphicalModel(
      (factor_modules): ModuleDict(
        (embedding): Identity()
        (c1): ProbEncoderFromEmb(
          (encoder): Sequential(
            (0): Linear(in_features=10, out_features=1, bias=True)
            (1): Unflatten(dim=-1, unflattened_size=(1,))
          )
        )
  

## 7. Training

We train the complete model with a combined loss:
- **Concept loss**: BCE loss between predicted and true concept labels (c1, c2)
- **Task loss**: BCE loss between predicted and true task labels (xor)
- **Total loss**: `concept_loss + concept_reg * task_loss`

Training process:
1. Compute embedding from input features
2. Query the inference engine with the embedding as evidence
3. Split predictions into concepts and tasks
4. Compute losses and backpropagate

In [55]:
# Setup training
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()
model.train()

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()

    # Compute embedding
    emb = encoder(x_train)
    
    # Inference: query the ProbabilisticModel with embedding as evidence
    cy_pred = inference_engine.query(query_concepts, evidence={'embedding': emb})
    
    # Split predictions: first columns are concepts, remaining are task
    c_pred = cy_pred[:, :c_train.shape[1]]
    y_pred = cy_pred[:, c_train.shape[1]:]

    # Compute loss
    concept_loss = loss_fn(c_pred, c_train)
    task_loss = loss_fn(y_pred, y_train)
    loss = concept_loss + concept_reg * task_loss

    # Backward pass
    loss.backward()
    optimizer.step()

    # Log progress
    if epoch % 100 == 0:
        task_accuracy = accuracy_score(y_train, y_pred > 0.)
        concept_accuracy = accuracy_score(c_train, c_pred > 0.)
        print(f"Epoch {epoch}: Loss {loss.item():.2f} | Task Acc: {task_accuracy:.2f} | Concept Acc: {concept_accuracy:.2f}")

print("\nTraining complete!")

Epoch 0: Loss 1.07 | Task Acc: 0.00 | Concept Acc: 0.22
Epoch 100: Loss 0.52 | Task Acc: 0.09 | Concept Acc: 0.97
Epoch 200: Loss 0.42 | Task Acc: 0.31 | Concept Acc: 0.99
Epoch 300: Loss 0.40 | Task Acc: 0.32 | Concept Acc: 0.99
Epoch 400: Loss 0.39 | Task Acc: 0.45 | Concept Acc: 0.99

Training complete!


## 8. Baseline Predictions (No Intervention)

Let's examine the model's predictions without any interventions.
The output contains concatenated predictions: [c1, c2, xor]

In [56]:
# Get baseline predictions
model.eval()
with torch.no_grad():
    emb = encoder(x_train)
    cy_pred = inference_engine.query(query_concepts, evidence={'embedding': emb})

print("Baseline predictions (first 5 samples):")
print("Format: [c1, c2, xor_class0, xor_class1]")
print(cy_pred[:5])
print(f"\nShape: {cy_pred.shape}")
print(f"  Columns 0-1: concept predictions (c1, c2)")
print(f"  Columns 2-3: task predictions (xor one-hot)")

Baseline predictions (first 5 samples):
Format: [c1, c2, xor_class0, xor_class1]
tensor([[-5.2508e+00,  1.9481e+01,  1.0985e-01, -1.1001e-01],
        [ 8.2998e+00,  4.2770e+00, -6.0167e-03,  7.3647e-03],
        [-1.4043e+01, -1.3596e+01,  4.0784e-02, -4.3052e-02],
        [-1.8641e+01,  1.6096e+01,  1.1045e-01, -1.1062e-01],
        [ 4.7895e+00,  9.1838e+00, -4.1456e-03,  5.5098e-03]])

Shape: torch.Size([1000, 4])
  Columns 0-1: concept predictions (c1, c2)
  Columns 2-3: task predictions (xor one-hot)


## 9. Interventions in BipartiteModel

The BipartiteModel framework supports interventions on the underlying ProbabilisticModel:
- Access the ProbabilisticModel's factor modules via `concept_model.probabilistic_model.cpd_modules`
- Apply interventions to specific factors (e.g., "c1.encoder")
- Effects propagate through the graph structure

### Intervention Setup:
- **Policy**: RandomPolicy to randomly select samples and intervene on concept c1
- **Strategy**: DoIntervention to set c1 to a constant value (-10)
- **Layer**: Intervene at the "c1.encoder" factor in the ProbabilisticModel
- **Quantile**: 1.0 (intervene on all selected samples)

In [57]:
# Compute embedding for intervention
emb = encoder(x_train)

# Create annotations for intervention
c_annotations = Annotations({1: AxisAnnotation(["c1"])})

# Define intervention policy and strategy
int_policy_c = UniformPolicy(
    out_annotations=c_annotations,
    subset=["c1"]
)
int_strategy_c = DoIntervention(
    model=concept_model.probabilistic_model.cpd_modules,
    constants=-10
)

print("Intervention configuration:")
print(f"  Policy: RandomPolicy on concept 'c1'")
print(f"  Strategy: DoIntervention with constant value -10")
print(f"  Target layer: c1.encoder (in BipartiteModel's ProbabilisticModel)")
print(f"  Quantile: 1.0 (intervene on all selected samples)")
print(f"\nThis intervention will:")
print(f"  1. Randomly select samples")
print(f"  2. Set concept c1 to -10 for those samples")
print(f"  3. Propagate the effect through the BipartiteModel to xor prediction")

Intervention configuration:
  Policy: RandomPolicy on concept 'c1'
  Strategy: DoIntervention with constant value -10
  Target layer: c1.encoder (in BipartiteModel's PGM)
  Quantile: 1.0 (intervene on all selected samples)

This intervention will:
  1. Randomly select samples
  2. Set concept c1 to -10 for those samples
  3. Propagate the effect through the BipartiteModel to xor prediction


## 10. Applying the Intervention

Now we apply the intervention and observe how the predictions change.
Compare these results with the baseline predictions above to see the intervention's effect.

In [58]:
print("Predictions with intervention:")
with intervention(
    policies=[int_policy_c],
    strategies=[int_strategy_c],
    on_layers=["c1.encoder"],
    quantiles=[1]
):
    cy_pred_intervened = inference_engine.query(query_concepts, evidence={'embedding': emb})
    print("Format: [c1, c2, xor_class0, xor_class1]")
    print(cy_pred_intervened[:5])

print("\nNote: Compare with baseline predictions above.")
print("You should see c1 values changed to -10 for randomly selected samples,")
print("and corresponding changes in the xor predictions.")

Predictions with intervention:
Format: [c1, c2, xor_class0, xor_class1]
tensor([[-10.0000,  19.4812,   0.1104,  -0.1106],
        [-10.0000,   4.2770,   0.1095,  -0.1097],
        [-10.0000, -13.5958,   0.0408,  -0.0430],
        [-10.0000,  16.0960,   0.1104,  -0.1106],
        [-10.0000,   9.1838,   0.1104,  -0.1106]], grad_fn=<SliceBackward0>)

Note: Compare with baseline predictions above.
You should see c1 values changed to -10 for randomly selected samples,
and corresponding changes in the xor predictions.


## Summary

In this notebook, we explored the BipartiteModel framework for concept-based learning:

1. **Data**: Loaded the XOR toy dataset with binary concepts
2. **Rich Annotations**: Defined metadata including distributions, types, and descriptions
3. **BipartiteModel**: High-level abstraction that automatically builds a ProbabilisticModel
4. **LazyConstructors**: Used to create encoder and predictor factors automatically
5. **Inference**: Queried the underlying ProbabilisticModel for predictions
6. **Training**: Trained with combined concept and task supervision
7. **Interventions**: Applied causal interventions via the ProbabilisticModel structure

### Key Advantages of BipartiteModel:
- **High-level abstraction**: Simplified ProbabilisticModel construction from annotations
- **Automatic structure**: Model builds Variables and ParametricCPDs automatically
- **Rich metadata**: Support for distributions, cardinalities, and descriptions
- **LazyConstructors**: Flexible way to specify encoder/predictor architectures
- **ProbabilisticModel access**: Full access to underlying ProbabilisticModel for advanced operations
- **Less boilerplate**: Reduces code needed compared to manual ProbabilisticModel construction

### Comparison with Other Approaches:
- **vs. Layer-based**: More structured, explicit graph representation
- **vs. Manual ProbabilisticModel**: Less code, automatic construction from metadata
- **Best for**: Production systems, complex models with many concepts/tasks

This framework is ideal for:
- Large-scale concept-based models with many variables
- Systems requiring rich metadata for interpretability
- Applications needing both ease-of-use and flexibility
- Production deployments with complex concept hierarchies